# 1-click deepfake

original repo: https://github.com/s0md3v/roop

`main` branch

In [ ]:
#@markdown **GPU check** (free T4, premium V100 or A100)
!nvidia-smi -L

In [ ]:
#@markdown **Setup environment**

%%bash

# pip install -r roop/requirements.txt
# pip uninstall onnxruntime
# pip install onnxruntime-gpu
pip install 'numpy>=1.23.5' customtkinter tkinterdnd2 onnxruntime-gpu opennsfw2 insightface gfpgan --progress-bar off

curl -sSL -o roop.zip https://github.com/s0md3v/roop/archive/main.zip
unzip -q -o roop.zip
rm roop.zip
mv roop-main roop

In [ ]:
#@markdown **download models weights**

%%bash

download_model () {
    wget -P $1 $2 --content-disposition --continue &> /dev/null
}

download_model roop/models https://huggingface.co/deepinsight/inswapper/resolve/main/inswapper_128.onnx
download_model ~/.opennsfw2/weights https://github.com/bhky/opennsfw2/releases/download/v0.1.0/open_nsfw_weights.h5
download_model ~/.insightface/models https://github.com/deepinsight/insightface/releases/download/v0.7/buffalo_l.zip

download_model roop/models https://github.com/TencentARC/GFPGAN/releases/download/v1.3.0/GFPGANv1.3.pth
download_model roop/models https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth
download_model gfpgan/weights https://github.com/xinntao/facexlib/releases/download/v0.1.0/detection_Resnet50_Final.pth
download_model gfpgan/weights https://github.com/xinntao/facexlib/releases/download/v0.2.2/parsing_parsenet.pth

cd ~/.insightface/models && unzip -q -o buffalo_l.zip -d buffalo_l

**ATTENTION:** after setup environment, you have to restart colab runtime

without it, the program will likely fail

In [ ]:
#@markdown **Swap face**

#@markdown files name shouldn’t contain non-latin characters

face_image = "photo-1535644258845-185656906071.jpeg"  # @param {type:"string"}
video_file = "yolo.webm"  # @param {type:"string"}
output_vid = "swapped.mp4"  # @param {type:"string"}
skip_audio = False  # @param {type:"boolean"}
keep_fps = True  # @param {type:"boolean"}
swap_all_faces = False  # @param {type:"boolean"}
#@markdown ↳ if disabled, only the leftmost face in each frame is replaced
enhacer_GFPGAN = True  # @param {type:"boolean"}
#@markdown ↳ if enable, enhance swapped faces but processing is slower
gpu_threads = 6  #@param {type:"integer"}
#@markdown ↳ to be decreased if crash / out of memory error<br />
#@markdown suggested value for colab free: 8 if disable enhancer, 6 if enable<br />
#@markdown if crash even with only 1 thread, try restarting runtime

skip_audio_opt = "--skip-audio" if skip_audio else ""
keep_fps_opt = "--keep-fps" if keep_fps else ""
all_faces_opt = "--many-faces" if swap_all_faces else ""
enhacer_opt = "face_swapper"
if enhacer_GFPGAN: enhacer_opt += " face_enhancer"

!python roop/run.py -s $face_image -t $video_file -o $output_vid $keep_fps_opt $all_faces_opt $skip_audio_opt --execution-provider cuda --execution-threads $gpu_threads --frame-processor $enhacer_opt